# Projekt MSP1
Cílem tohoto projektu je se seznámit s programovými nástroji využívaných ve statistice a osvojit si základní procedury. Projekt není primárně zaměřen na efektivitu využívání programového vybavení (i když úplně nevhodné konstrukce mohou mít vliv na hodnocení), ale nejvíce nás zajímají vaše statistické závěry a způsob vyhodnocení. Dbejte také na to, že každý graf musí splňovat nějaké podmínky - přehlednost, čitelnost, popisky.

V projektu budete analyzovat časy běhu šesti různých konfigurací algoritmů. Ke každé konfiguraci vzniklo celkem 200 nezávislých běhů, jejichž logy máte k dispozici v souboru [logfiles.zip](logfiles.zip).

Pokud nemáte rozchozené prostředí pro pro spouštění Jupyter notebooku, můžete využití službu [Google Colab](https://colab.google/). Jakákoliv spolupráce, sdílení řešení a podobně je zakázána!

S případnými dotazy se obracejte na Vojtěcha Mrázka (mrazek@fit.vutbr.cz).

__Odevzdání:__ tento soubor (není potřeba aby obsahoval výstupy skriptů) do neděle 22. 10. 2023 v IS VUT. Kontrola bude probíhat na Pythonu 3.10.12; neočekává se však to, že byste používali nějaké speciality a nekompatibilní knihovny. V případě nesouladu verzí a podobných problémů budete mít možnost reklamace a prokázání správnosti funkce. Bez vyplnění vašich komentářů a závěrů do označených buněk nebude projekt hodnocen!

__Upozornění:__ nepřidávejte do notebooku další buňky, odpovídejte tam, kam se ptáme (textové komentáře do Markdown buněk)

__Tip:__ před odevzdáním resetujte celý notebook a zkuste jej spustit od začátku. Zamezíte tak chybám krokování a editací, kdy výsledek z buňky na konci použijete na začátku.

__OTÁZKA K DOPLNĚNÍ:__

_ Jméno a login autora_

David Chocholatý, xchoch09

## Načtení potřebných knihoven
Načtěte knihovny, které jsou nutné pro zpracování souborů a práci se statistickými funkcemi. Není dovoleno načítat jiné knihovny.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from zipfile import ZipFile

## Načtení dat do DataFrame
Ze souboru `logfiles.zip` umístěném ve stejném adresáři načtěte data a vytvořte Pandas DataFrame.

Z logu vás budou nejvíce zajímat řádky
```
Configuration: config6
Run: 191
Time of run: 53.298725254089774
```

Můžete využít následující kostru - je vhodné pracovat přímo se ZIP souborem. Jedinou nevýhodou je to, že vám bude vracet _byte_ objekt, který musíte přes funkci `decode` zpracovat.

In [ ]:
def load_logfile(f) -> dict:
    """Load a logfile from a file-like object and return a dict with the data."""
    data = {
        "conf": None,
        "run": None,
        "time": np.nan
    }
    
    for line in f:
        line = line.decode("utf-8")

        # Odstranění znaku nového řádku a rozdělení řádku na páry klíč–hodnota.
        [line_key, line_value] = line.strip().split(": ")                                
        
        if line_key == "Configuration":
            data["conf"] = line_value
        elif line_key == "Run":
            data["run"] = int(line_value)
        elif line_key == "Time of run":            
            data["time"] = float(line_value)

    return data

data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        with zf.open(filename, "r") as f:
            data.append(load_logfile(f))

df = pd.DataFrame(data)
df

## Analýza a čištění dat
Vhodným způsobem pro všechny konfigurace analyzujte časy běhů a pokud tam jsou, identifikujte hodnoty, které jsou chybné. 

In [ ]:
plt.figure(figsize=(12,10))
for figid, (conf, d) in enumerate(df.groupby("conf")):
    ax = plt.subplot(321 + figid)
    ax.hist(d["time"], label=conf, bins=200)
    ax.set_title(conf)
    ax.set(xlabel="Čas", ylabel="Četnost")

plt.tight_layout()
###################

xlim_val = plt.xlim() # Uložení xlim hodnot (min, max) pro zobrazení grafů po odstranění chybných dat.

__OTÁZKA K DOPLNĚNÍ:__

_Objevily se nějaké chybné hodnoty? Proč tam jsou s ohledem na to, že se jedná o běhy algoritmů?_

Ano. V datech jsou uloženy také chybné hodnoty. Konkrétně lze označit hodnoty času (time) 0.01 a 3600 pro jednotlivé běhy (run) konfigurací (conf) za chybné.

<u>Hodnota 0.01</u>:

Zaprvé četnost zastoupení této časové hodnoty v datech je oproti ostatním hodnotam minimální. Zároveň tato časová hodnota je velmi _malá_ oproti dalším časovým hodnotám a hodnoty jí blízké nejsou v datech jinak zastoupeny.

Na základě skutečnosti, že data zachycují jednotlivé běhy algoritmů, se s největší pravděpodobností jedná o ukončení programu s chybovým stavem, tudíž celý algoritmus nebyl proveden a byl předčasně ukončen. Při bližší analýze dat může být tato skutečnost potvrzena, že všechny běhy, jejichž čas odpovídal dané hodnotě, skončily s chybovým stavem SEGFAULT.

<u>Hodnota 3600</u>:

Zaprvé četnost zastoupení této časové hodnoty v datech je oproti ostatním hodnotam minimální. Zároveň tato časová hodnota je velmi _velká_ oproti dalším časovým hodnotám a hodnoty jí blízké nejsou v datech jinak zastoupeny.

Na základě skutečnosti, že data zachycují jednotlivé běhy algoritmů, se s největší pravděpodobností jedná o ukončení programu na základě překročení času omezujícího dobu povolenou pro běh algoritmu (timeout) a došlo k nevalidnímu ukončení co se týče běhu algoritmu. Při bližší analýze dat může být tato skutečnost potvrzena, že všechny běhy, jejichž čas odpovídal dané hodnotě, skončily s chybovým stavem TIME LIMIT.

Vyčistěte dataframe `df` tak, aby tam tyto hodnoty nebyly a ukažte znovu analýzu toho, že čištění dat bylo úspěšné. Odtud dále pracujte s vyčištěným datasetem.

In [ ]:
# Vyčištění chybných hodnot (0.01 a 3600)
df = df[(df.time > 0.01) & (df.time < 3600)]

# Znovuvykreslení dat    
plt.figure(figsize=(12,10))
plt.suptitle(" Grafy pro ověření správnosti čištění dat: ", fontsize=20)
for figid, (conf, d) in enumerate(df.groupby("conf")):
    ax = plt.subplot(321 + figid)
    ax.hist(d["time"], label=conf, bins=200)
    ax.set_title(conf)   
    ax.set(xlabel="Čas", ylabel="Četnost")
    ax.set_xlim(xlim_val)

plt.tight_layout()

plt.figure(figsize=(12,10))
plt.suptitle(" Zobrazení dat v přívětivější podobě: ", fontsize=20)
for figid, (conf, d) in enumerate(df.groupby("conf")):
    ax = plt.subplot(321 + figid)
    ax.hist(d["time"], label=conf, bins=30)
    ax.set_title(conf)
    ax.set(xlabel="Čas", ylabel="Četnost")
    ax.set_xlim(0, 300)

plt.tight_layout()


## Deskriptivní popis hodnot
Vypište pro jednotlivé konfigurace základní deskriptivní parametry času pro jednotlivé konfigurace.  

__TIP__ pokud výsledky uložíte jako Pandas DataFrame, zobrazí se v tabulce.

In [ ]:
df_grouped = df.groupby("conf")["time"]

print("Koeficient šikmosti")
print("###################")
print(df_grouped.skew())
print()

print("Koeficient špičatosti")
print("###################")
print(df_grouped.apply(pd.DataFrame.kurt))
print()

print("Další základní deskriptivní parametry")
print("######################################")
df_grouped.describe()

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte, co všechno můžeme z parametrů vyčíst._

<u>count</u>:

Nejprve si můžeme povšimnout, že na základě hodnoty "count" každá z konfigurací měla nějaký "run", kdy skončila chybovým stavem nebo běh překročil maximální časový limit. Nejmenší chybovosti dosahuje konfigurace *config4* a největší konfigurace *config1*.

<u>mean</u>:

Na základě získaných a pozorovaných dat se mezi konfigurace s nejmenší průměrnou dobou běhu řadí konfigurace *config1* a *config4*, přičemž konfigurace *config1* má nejměnší průměrnou hodnotu doby běhu algoritmu.

Dále mezi konfigurace s největší průměrnou dobou běhu se řadí konfigurace *config5* a *config6*, přičemž konfigurace *config5* má největší průměrnou hodnotu doby běhu algoritmu.

<u>std</u>:

Z dat směrodatných odchylek všech konfigurací vyplývá, že rozptyl naměřených hodnot je nejmenší pro konfiguraci *config1* a největší pro konfiguraci *config4*.

<u>min</u>:

V datech je zachyceno, že nejmenší časovou hodnotu doby běhu algoritmu mají konfigurace *config4* a *config1*. Na druhé straně konfigurace *config5* a *config6* mají nejvyšší minimální dobu běhu.

<u>25%, 50%, 75%</u>:

Dle hodnot percentilů pro jednotlivé konfigurace lze určit, že nejmenší časové hodnoty, ve kterých je 25%, 50% a 75% dat, mají konfigurace *config1* a *config4*. Naopak nejvyšší hodnoty mají konfigurace *config5* a *config6*.

<u>max</u>:

Z pozorování dat vyplývá, že v parametru nejdelší doby validně ukončeného běhu konfigurace dominuje konfigurace *config1* s nejmenší časovou hodnotou. Naopak na druhé straně škály se nachází konfigurace *config5*.

<u>Koeficient šikmosti</u>:

Hodnota koeficientu určuje symetričnost dat. Zároveň pro normální rozdělení by mělo platit, že hodnota je blízká hodnotě 0, což lze ze získaných dat potvrdit.

<u>Koeficient špičatosti</u>:

Na základě hodnot koeficientu šikmosti lze pomocí hodnot koeficientu špičatosti blízkých 0 rozhodnout, že ve všech případech konfigurací se jedná o normální rozdělení.

Sumarizace:
Na základě analýzy dat lze usoudit, že konfigurace *config1* a *config4* mohou být potenciálně nejlepší, co se týče časové optimálnosti algoritmu. Naopak potenciálně nejhorší jsou konfigurace *config5* a *config6*. S využitím hodnot koeficientů šikmosti a špičatosti lze tvrdit, že pro všechny konfigurace se jedná o normální rozdělení.

## Vizualizace
Vizualizujte časy běhů algoritmů v jednom kompaktním grafu tak, aby byl zřejmý i rozptyl hodnot. Zvolte vhodný graf, který pak níže komentujte.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.boxplot(df.groupby("conf")["time"].apply(list), labels=df.conf.unique())
ax.set(xlabel="Konfigurace", ylabel="Čas", ylim=(0, None))
ax.set_title("Časy běhů algoritmů")

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte  výsledky z tabulky._

Z vizualizace ve formě Boxplot lze potvrdit na základě minimálních hodnot, mediánu a interkvartilního rozsahu, že pro bližší zkoumání by měly být vybrány konfigurace *config1* a *config4*. Zároveň lze pozorovat, že hodnoty interkvartilního rozsahu a mediánu jsou nejmenší pro konfiguraci *config1* a lze prozatím předpokládat, že se jedná o nejoptimálnější konfiguraci.

Na základě rozdílné "výšky boxů" v grafu můžeme určit, že pro určení efektivity konfigurací bude vhodné použit nepárový T-test (Welchův test).

## Určení efektivity konfigurací algoritmů
Nás ale zajímá, jaká konfigurace je nejrychlejší. Z výše vykresleného grafu můžeme vyloučit některé konfigurace. Existuje tam však minimálně jedna dvojice, u které nedokážeme jednoznačně určit, která je lepší - pokud nebudeme porovnávat pouze extrémní hodnoty, které mohou být dané náhodou, ale celkově. Proto proveďte vhodný test významnosti - v následující části diskutujte zejména rozložení dat (i s odkazem na předchozí buňky, variabilitu vs polohu a podobně). Je nutné každý logický krok a výběry statistických funkcí komentovat. Můžete i přidat další buňky.

Vužijte vhodnou funkci z knihovny `scipy.stats` a funkci poté __implementujte sami__ na základě základních matematických funkcí knihovny `numpy` případně i funkcí pro výpočet studentova rozložení v [scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html). Při vlastní implementaci není nutné se primárně soustředit na efektivitu výpočtu (není potřeba využít všechny funkce numpy, můžete použít normální cykly a podobně - v hodnocení však bude zahrnuta přehlednost a neměly by se objevit jasné chyby, jako je zvýšení třídy složitosti a podobně).

__OTÁZKA K DOPLNĚNÍ:__

_Jaká data budete zkoumat? Jaké mají rozložení a parametry (např. varianci) a jaký test použijete? Jaká je nulová hypotéza? Jak se liší variabilita a poloha vybraných konfigurací?_

Na základě předchozích analýz bylo rozhodnuto, že pro další zkoumání budou vybrány konfigurace *config1* a *config4*. Bylo zjištěno, že ve všech případech se jedná o normální rozdělení. Jak již bylo uvedeno, použijeme nepárový T-test, a to konkrétně Welchův test. Volbu tohoto testu lze argumentovat rozdílnou variancí (**variabilita**) dat pro vybrané konfigurace. Na základě skutečnosti, že medián (**poloha**) hodnot pro konfiguraci *config1* je menší než medián hodnot pro konfiguraci *config4*, bude nulová a alternativní hypotéza stanovena následovně.

<u>Nulová hypotéza</u>

Oba vzorky musí mít stejnou průměrnou hodnotu.

<u>Alternativní hypotéza</u>

Průměrná hodnota distribuce pro *config1* je menší než průměrná hodnota distribuce pro *config4*.

In [ ]:
config_1 = df[(df.conf == "config1")]["time"]
config_4 = df[(df.conf == "config4")]["time"]

statistical_significance = 0.05

print("T-test")
print("####################################")

# Alternative (less): config1 < config4
result_ttest = stats.ttest_ind_from_stats(config_1.mean(), config_1.std(), len(config_1),
                                          config_4.mean(), config_4.std(), len(config_4),
                                          equal_var=False, alternative="less")

print("statistic =", result_ttest.statistic)
print("pvalue =", result_ttest.pvalue)
print()

if result_ttest.pvalue < statistical_significance:
    print("Nulová hypotéza se zamítá a platí alternativní hypotéza.")
else:
    print("Nulová hypotéza se nezamítá.")

__OTÁZKA K DOPLNĚNÍ:__

_Jaký je závěr statistického testu?_

Na základě výsledku testu zamítáme alternativní hypotézu a platí alternativní hypotéza. Celkovým výsledkem bylo stanoveno, že konfigurace *config1* je efektivnější než konfigurace *config4*. Zároveň dle výsledků předchozí analýzy vyplývá, že konfigurace *config1* je nejefektivnější konfigurací ze všech zkoumaných.

In [ ]:
# https://en.wikipedia.org/wiki/Student%27s_t-test#Independent_two-sample_t-test
def independent_two_sample_ttest(s_1, nobs_1, s_2, nobs_2):        
    degrees_of_freedom = nobs_1 + nobs_2 - 2.0
    svar = ((nobs_1 - 1) * s_1 + (nobs_2 - 1) * s_2) / degrees_of_freedom
    divisor = np.sqrt(svar * (1.0 / nobs_1 + 1.0 / nobs_2))
    
    return degrees_of_freedom, divisor

# https://en.wikipedia.org/wiki/Welch%27s_t-test
def welchs_ttest(s_1, nobs_1, s_2, nobs_2):
    sn_1 = s_1 / nobs_1
    sn_2 = s_2 / nobs_2
        
    try:
        degrees_of_freedom = (sn_1 + sn_2)**2 / (sn_1**2 / (nobs_1 - 1) + sn_2**2 / (nobs_2 - 1))
    except ZeroDivisionError:
        print("Info: Division by zero, setting to 0.0.")        
        degrees_of_freedom = 0.0

    divisor = np.sqrt(sn_1 + sn_2)
    
    return degrees_of_freedom, divisor

def own_ttest_ind_from_stats(mean_1, std_1, nobs_1, mean_2, std_2, nobs_2,
                             equal_var=True, alternative="two-sided"):
    mean_1 = np.asarray(mean_1)
    std_1 = np.asarray(std_1)
    mean_2 = np.asarray(mean_2)
    std_2 = np.asarray(std_2)

    s_1 = std_1**2
    s_2 = std_2**2
    
    if equal_var:
        degrees_of_freedom, divisor = independent_two_sample_ttest(s_1, nobs_1, s_2, nobs_2)
    else:
        degrees_of_freedom, divisor = welchs_ttest(s_1, nobs_1, s_2, nobs_2)
    
    diff = (mean_1 - mean_2)
    
    if divisor == 0.0:
        print("Info: Division by zero, setting to 0.0.")        
        t = 0.0
    else:
        t = np.divide(diff, divisor)    
    
    if alternative == "two-sided":
        p_value = stats.t.cdf(-np.abs(t), degrees_of_freedom)*2
    elif alternative == "less":
        p_value = stats.t.cdf(t, degrees_of_freedom)        
    elif alternative == "greater":
        p_value = stats.t.cdf(-t, degrees_of_freedom)   
    else:
        raise ValueError("(alternative) - one of the following must to be provided: two-sided, less or greater")
    
    return t, p_value

print("T-test")
print("####################################")

t, p_value = own_ttest_ind_from_stats(config_1.mean(), config_1.std(), len(config_1), config_4.mean(), config_4.std(), len(config_4), equal_var=False, alternative="less")

print("statistic =", t)
print("pvalue =", p_value)